In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("./input_data/inflow_by_prof.csv")
print(data.describe())
print(data.head())
print(data['label+1'].value_counts())

        Unnamed: 0        value      value-1      value-2      value-3  \
count  1547.000000  1547.000000  1547.000000  1547.000000  1547.000000   
mean    773.000000     7.591629     7.595048     7.594163     7.592902   
std     446.724747     7.025416     7.024501     7.025085     7.026004   
min       0.000000     0.800000     0.800000     0.800000     0.800000   
25%     386.500000     2.805000     2.805000     2.800000     2.800000   
50%     773.000000     4.440000     4.460000     4.460000     4.460000   
75%    1159.500000    10.310000    10.310000    10.310000    10.310000   
max    1546.000000    32.790000    32.790000    32.790000    32.790000   

           value-4      value-5      value-6      value-7      value-8  \
count  1547.000000  1547.000000  1547.000000  1547.000000  1547.000000   
mean      7.591765     7.590692     7.589405     7.588707     7.587453   
std       7.026837     7.027572     7.028305     7.028561     7.028585   
min       0.800000     0.800000     0

## My mean threshold method

In [4]:
data = pd.read_csv("./input_data/inflow_by_mean.csv")
data.head()

,Unnamed: 0,time,value,value-1,value-2,value-3,value-4,value-5,value-6,value-7,...,value-9,value-10,value-11,value-12,month_text,month+1_text,month,month+1,label,label+1
0,0,8-1-1872,23.60,8.36,2.28,1.50,1.57,1.95,2.59,4.48,...,9.13,17.60,25.80,24.00,August,September,8,9,Medium High,Medium High
1,1,9-1-1872,27.30,23.60,8.36,2.28,1.50,1.57,1.95,2.59,...,6.48,9.13,17.60,25.80,September,October,9,10,Medium High,High
2,2,10-1-1872,21.30,27.30,23.60,8.36,2.28,1.50,1.57,1.95,...,4.48,6.48,9.13,17.60,October,November,10,11,High,High
3,3,11-1-1872,12.90,21.30,27.30,23.60,8.36,2.28,1.50,1.57,...,2.59,4.48,6.48,9.13,November,December,11,12,High,High
4,4,12-1-1872,8.31,12.90,21.30,27.30,23.60,8.36,2.28,1.50,...,1.95,2.59,4.48,6.48,December,January,12,1,High,High


In [5]:
data['label+1'].value_counts()

Medium High    541
High           536
Medium         248
Medium Low     118
Low            104
Name: label+1, dtype: int64

In [9]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

In [ ]:
# load the dataset
def load_dataset(filename, cols=None):
    # load the dataset as a pandas DataFrame
    data = pd.read_csv(filename, usecols=cols)
    # retrieve numpy array
    dataset = data.values
    # split into input (X) and output (y) variables
    X = dataset[:, :-1]
    y = dataset[:,-1]
    return X, y

# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc

# feature selection
def select_reg_features(X_train, y_train, X_test, test_func="ANOVA", k_type='all'):
    if test_func == "ANOVA":
        score_func = f_classif
    elif test_func == "MI":
        score_func = mutual_info_classif
    # configure to select all features
    fs = SelectKBest(score_func=score_func, k=k_type)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

# show results 
def show_results(scores, x_ticks, filename, x_label="Features", title="Test", figsize=(8, 4), verbose=True):
    plt.figure(figsize=figsize)
    
    # what are scores for the features
    for idx, score in enumerate(scores):
        print(f'Feature {x_ticks[idx]}: {score}')
        
    # plot the scores
    plt.bar(x_ticks, scores)
    plt.xticks(rotation=45, ha='right')
    plt.xlabel(x_label)
    plt.title(title)
    plt.savefig(filename, bbox_inches='tight')
    if verbose:
        plt.show()


# split into input (X) and output (y) variables
cols = ['value', 'value-1', 'value-2', 'value-3', 'value-4', 'value-5',
                 'value-6', 'value-7', 'value-8', 'value-9', 'value-10', 'value-11', 'value-12', "label+1"]
cols_name_saved = ['t', 't-1', 't-2', 't-3', 't-4', 't-5', 't-6', 't-7', 't-8', 't-9', 't-10', 't-11', 't-12']
X, y = load_dataset("./input_data/inflow_by_mean.csv", cols)

# split into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=20)

# prepare output data
y_train, y_test = prepare_targets(Y_train, Y_test)

print(X_train[1])
print(y_train[1])

In [ ]:
# feature selection using ANOVA-test 

# feature selection using ANOVA-test 
X_train_fs, X_test_fs, fs = select_reg_features(X_train, y_train, X_test, "ANOVA", 'all')

# show results
show_results(fs.scores_, cols_name_saved, "anova.png", x_label="Features", title="ANOVA test", figsize=(7, 4), verbose=True)


In [ ]:
# feature selection using mutual information
X_train_fs, X_test_fs, fs = select_reg_features(X_train, y_train, X_test, "MI", 'all')

# show results
show_results(fs.scores_, cols_name_saved, "mi.png", x_label="Features", title="MI test", figsize=(7, 4), verbose=True)
